# 🎬 Movie Revenue Prediction Pipeline Preview

Notebook này minh họa toàn bộ quy trình dự án (End-to-End Pipeline):
1. **Data Loading**: Tải dữ liệu thô.
2. **Preprocessing**: Làm sạch, xử lý outlier, và tạo features (TF-IDF, encoding).
3. **Model Training**: Huấn luyện và tối ưu các mô hình (RandomForest, XGBoost, LightGBM).
4. **Evaluation**: So sánh kết quả.

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# 1. Setup đường dẫn project
# Giả sử notebook đang chạy trong folder 'notebooks'
current_dir = Path(os.getcwd())
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
else:
    project_root = current_dir

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"Project Root: {project_root}")

# Import các modules từ src
from src.data_loader import TMDbDataLoader
from src.preprocessing import DataPreprocessor
from src.model_trainer import ModelTrainer

Project Root: d:\Movie_Revenue_Prediction_v2


c:\Users\PC\anaconda3\envs\movie_v2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Data Loading
Tải dữ liệu raw từ file CSV hoặc fetch mới từ API nếu cần.

In [ ]:
config_path = project_root / "configs" / "config.yaml"
raw_data_path = project_root / "data" / "raw" / "movies_2020_2024.csv"

# Khởi tạo DataLoader
loader = TMDbDataLoader(config_path=str(config_path))

# Load dữ liệu (nếu chưa có file thì fetch)
if not raw_data_path.exists():
    print("Chưa thấy file raw, đang fetch dữ liệu từ API...")
    loader.fetch_data()
    loader.save_data(str(raw_data_path))
else:
    print("Đã tìm thấy file raw data.")

df_raw = loader.load_data(str(raw_data_path))
print(f"Raw Data Shape: {df_raw.shape}")
df_raw.head(3)

2025-12-07 15:59:30,745 - src.data_loader - INFO - TMDbDataLoader đã được khởi tạo thành công
2025-12-07 15:59:30,747 - src.data_loader - INFO - Fetch phim từ 2010–2024
2025-12-07 15:59:30,748 - src.data_loader - INFO - Đang fetch phim năm 2010...


Chưa thấy file raw, đang fetch dữ liệu từ API...


2025-12-07 15:59:58,632 - src.data_loader - INFO - Đang fetch phim năm 2011...
2025-12-07 16:00:33,229 - src.data_loader - INFO - Đang fetch phim năm 2012...
2025-12-07 16:01:02,206 - src.data_loader - INFO - Đang fetch phim năm 2013...
2025-12-07 16:01:34,360 - src.data_loader - INFO - Đang fetch phim năm 2014...
2025-12-07 16:02:05,487 - src.data_loader - INFO - Đang fetch phim năm 2015...
2025-12-07 16:02:37,854 - src.data_loader - INFO - Đang fetch phim năm 2016...
2025-12-07 16:03:06,114 - src.data_loader - INFO - Đang fetch phim năm 2017...
2025-12-07 16:03:34,867 - src.data_loader - INFO - Đang fetch phim năm 2018...
2025-12-07 16:04:06,069 - src.data_loader - INFO - Đang fetch phim năm 2019...
2025-12-07 16:04:35,401 - src.data_loader - INFO - Đang fetch phim năm 2020...


## 2. Data Preprocessing
Bước này sẽ:
- Xử lý missing values
- Loại bỏ outliers (dựa trên config IQR)
- Tạo features từ ngày tháng
- One-hot encoding cho Genres
- TF-IDF cho Overview
- Scaling dữ liệu

In [16]:
# Khởi tạo Preprocessor
preprocessor = DataPreprocessor(config_path=str(config_path))

# Fit và Transform dữ liệu
# Lưu ý: fit_transform sẽ loại bỏ outliers đối với tập train
X, y = preprocessor.fit_transform(df_raw)

print(f"\nProcessed Data Shape (X): {X.shape}")
print(f"Target Shape (y): {y.shape}")

# Giải thích số lượng features
feature_names = preprocessor.get_feature_names()
print(f"\nTổng số features: {len(feature_names)}")
print("Ví dụ 10 features đầu: ", feature_names[:10])
print("Ví dụ features TF-IDF: ", [f for f in feature_names if 'tfidf' in f][:5])

2025-12-07 15:41:52,374 - src.preprocessing - INFO - DataPreprocessor đã được khởi tạo thành công (target_col=revenue, scaler=StandardScaler)
2025-12-07 15:41:52,389 - src.preprocessing - INFO - Bắt đầu fit preprocessing pipeline...
d:\Movie_Revenue_Prediction_v2\src\preprocessing.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('', inplace=True)
d:\Movie_Revenue_Prediction_v2\src\preprocessing.py:157: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment u


Processed Data Shape (X): (433, 134)
Target Shape (y): (433,)

Tổng số features: 134
Ví dụ 10 features đầu:  ['budget', 'runtime', 'vote_average', 'vote_count', 'popularity', 'release_year', 'release_month', 'release_quarter', 'release_day_of_week', 'is_weekend']
Ví dụ features TF-IDF:  ['tfidf_0', 'tfidf_1', 'tfidf_2', 'tfidf_3', 'tfidf_4']


### Lưu dữ liệu đã xử lý
Lưu lại để có thể dùng cho training sau này mà không cần chạy lại preprocessing.

In [17]:
processed_dir = project_root / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

pd.DataFrame(X, columns=feature_names).to_csv(processed_dir / "X_train.csv", index=False)
pd.DataFrame(y, columns=['revenue']).to_csv(processed_dir / "y_train.csv", index=False)

print("Đã lưu processed data.")

Đã lưu processed data.


## 3. Model Training
Huấn luyện và so sánh 3 mô hình: RandomForest, XGBoost, LightGBM.

In [18]:
# Khởi tạo Trainer
trainer = ModelTrainer(config_path=str(config_path))

# Chia tập Train/Test (Split data)
trainer.split_data(X, y)

# Train tất cả mô hình (có kèm tối ưu Hyperparameters với Optuna)
# Lưu ý: Có thể mất vài phút tùy vào cấu hình n_trials trong config.yaml
trainer.train_all_models()

2025-12-07 15:42:17,311 - src.model_trainer - INFO - ModelTrainer đã được khởi tạo thành công
2025-12-07 15:42:17,331 - src.model_trainer - INFO - Đã split data - Train: (346, 134), Test: (87, 134)
2025-12-07 15:42:17,334 - src.model_trainer - INFO - 
2025-12-07 15:42:17,335 - src.model_trainer - INFO - BẮT ĐẦU TRAINING TẤT CẢ MODELS
2025-12-07 15:42:17,337 - src.model_trainer - INFO - ============================================================

2025-12-07 15:42:17,339 - src.model_trainer - INFO - 
2025-12-07 15:42:17,340 - src.model_trainer - INFO - Training RANDOM_FOREST
2025-12-07 15:42:17,341 - src.model_trainer - INFO - ============================================================
2025-12-07 15:42:17,342 - src.model_trainer - INFO - Bắt đầu optimize RandomForest với Optuna...
Best trial: 47. Best value: 1.52979: 100%|██████████| 50/50 [02:32<00:00,  3.06s/it, 152.75/3600 seconds]
2025-12-07 15:44:51,016 - src.model_trainer - INFO - RandomForest - Best RMSE: 1.53
2025-12-07 15:44:5

## 4. Evaluation & Comparison
So sánh hiệu năng các mô hình dựa trên RMSE, MAE, R2.

In [19]:
comparison_df = trainer.compare_models()
comparison_df

2025-12-07 15:46:47,181 - src.model_trainer - INFO - 
2025-12-07 15:46:47,181 - src.model_trainer - INFO - SO SÁNH MODELS
2025-12-07 15:46:47,184 - src.model_trainer - INFO - ============================================================


                       RMSE           MAE        R2        MAPE
xgboost        2.886927e+07  1.721451e+07  0.193003  404.947738
random_forest  2.944880e+07  1.715666e+07  0.160278  405.278293
lightgbm       3.046818e+07  1.823865e+07  0.101137  379.770542


,RMSE,MAE,R2,MAPE
xgboost,2.886927e+07,1.721451e+07,0.193003,404.947738
random_forest,2.944880e+07,1.715666e+07,0.160278,405.278293
lightgbm,3.046818e+07,1.823865e+07,0.101137,379.770542


In [20]:
# Lưu kết quả và Model tốt nhất
trainer.save_results()
trainer.save_all_models()
print("Hoàn tất pipeline!")

2025-12-07 15:47:09,509 - src.model_trainer - INFO - Đã lưu kết quả vào: results/experiments.csv
2025-12-07 15:47:09,611 - src.model_trainer - INFO - Đã lưu random_forest vào: models/random_forest.pkl
2025-12-07 15:47:09,622 - src.model_trainer - INFO - Đã lưu xgboost vào: models/xgboost.pkl
2025-12-07 15:47:09,644 - src.model_trainer - INFO - Đã lưu lightgbm vào: models/lightgbm.pkl


Hoàn tất pipeline!
